In [1]:
import pandas as pd
import numpy as np
import h5py
import geopandas as gpd
from scipy.spatial import cKDTree
# import plotly.express as px
import plotly.graph_objects as go
import json

In [13]:
hdf_file.close()

In [8]:
# from geojson HWM file, extract the coordinates of the reference points.
ref_points_gdf = gpd.read_file(r"V:\projects\p00832_ocd_2023_latz_hr\01_processing\GIS\Collected_HWMs\LA_high_water_mark_data_USGS\Laura 2020 4326.geojson")
# ref_points_gdf.head()
# drop all columns except: ["geometry", "site_no", "elev_ft", "hwmQuality", "verticalDa"]
ref_points_gdf = ref_points_gdf[["geometry", "site_no", "elev_ft", "hwmQuality", "verticalDa"]]
ref_points_gdf.head()

,geometry,site_no,elev_ft,hwmQuality,verticalDa
0,POINT (-89.72722 30.17611),LASTT23298,3.1,Excellent: +/- 0.05 ft,NAVD88
1,POINT (-93.46350 29.76990),LACAM27066,9.5,Good: +/- 0.10 ft,NAVD88
2,POINT (-90.24611 29.34194),LALAF27100,4.0,Good: +/- 0.10 ft,NAVD88
3,POINT (-91.85500 29.73389),LASTM27175,9.0,Fair: +/- 0.20 ft,NAVD88
4,POINT (-92.90602 29.75152),LACAM30158,11.7,Good: +/- 0.10 ft,NAVD88


In [172]:
# from RAS HDF file, extract the coordinates of the reference points.
hdf_fn = r"S:\For_Myles\LWI\HDFs_01.31.2024\10min_VS_5minComparisons\LWI_Coastwide_TZ_RASV61.p02.hdf"
hdf_file = h5py.File(hdf_fn, 'r')

ref_points = hdf_file['Geometry/Reference Points']
# ref_points = pd.DataFrame(ref_points[...], columns=['X', 'Y'])
# print(ref_points.keys())
# print(ref_points['Attributes']['Name'])
ref_points_names = [x.decode('utf-8') for x in ref_points['Attributes']['Name'][...]]
ref_points_coords = ref_points['Points'][...]
# Get Projection of the coordinate data
proj = hdf_file.attrs['Projection'].decode('utf-8')

ref_points_df = pd.DataFrame(ref_points_coords, columns=['X', 'Y'], index=ref_points_names)
ref_points_df


,X,Y
Reference Point 200,4.170729e+06,661668.883684
Reference Point 199,3.966316e+06,639864.068665
Reference Point 198,3.931669e+06,537467.475153
Reference Point 197,3.870530e+06,455032.911969
Reference Point 196,3.917106e+06,286591.621196
...,...,...
Reference Point 5,3.487089e+06,710983.643722
Reference Point 4,3.553495e+06,712586.538007
Reference Point 3,3.570462e+06,699419.906387
Reference Point 2,3.863242e+06,667453.614663


In [173]:
# pull out the hdf geometry to a geodataframe.
gdf_hdf = gpd.GeoDataFrame(ref_points_df, geometry=gpd.points_from_xy(ref_points_df.X, ref_points_df.Y), crs=proj)
gdf_hdf.reset_index(inplace=True)
gdf_hdf.rename(columns={'index':'Name'}, inplace=True)
# drop the X and Y columns
gdf_hdf.drop(columns=['X', 'Y'], inplace=True)
gdf_hdf.to_crs(4326, inplace=True)
gdf_hdf

,Name,geometry
0,Reference Point 200,POINT (-88.51347 30.28935)
1,Reference Point 199,POINT (-89.16234 30.24164)
2,Reference Point 198,POINT (-89.27786 29.96188)
3,Reference Point 197,POINT (-89.47514 29.73810)
4,Reference Point 196,POINT (-89.33764 29.27282)
...,...,...
195,Reference Point 5,POINT (-90.67874 30.45343)
196,Reference Point 4,POINT (-90.46796 30.45663)
197,Reference Point 3,POINT (-90.41445 30.42006)
198,Reference Point 2,POINT (-89.48732 30.32246)


In [10]:
import folium
m = folium.Map([30, -91], zoom_start=8, tiles="cartodbpositron")
folium.GeoJson(ref_points_gdf,
               tooltip=folium.GeoJsonTooltip(
                    fields=["elev_ft"])).add_to(m)

m

In [11]:
# from RAS HDF file, extract the coordinates of the reference points.
hdf_fn = r"S:\For_Myles\LWI\HDFs_01.31.2024\10min_VS_5minComparisons\LWI_Coastwide_TZ_RASV61.p02.hdf"
hdf_file = h5py.File(hdf_fn, 'r')

# Get Projection of the coordinate data
proj = hdf_file.attrs['Projection'].decode('utf-8')

# Get start time
timestamps = [t.decode('utf-8') for t in hdf_file['/Results/Unsteady/Output/Output Blocks/Base Output/Unsteady Time Series/Time Date Stamp']]
startTime = timestamps[0]

# Get time data referenced as days since start time.
time = np.array(hdf_file['/Results/Unsteady/Output/Output Blocks/Base Output/Unsteady Time Series/Time'])
timestamps

['20AUG2020 00:00:00',
 '20AUG2020 01:00:00',
 '20AUG2020 02:00:00',
 '20AUG2020 03:00:00',
 '20AUG2020 04:00:00',
 '20AUG2020 05:00:00',
 '20AUG2020 06:00:00',
 '20AUG2020 07:00:00',
 '20AUG2020 08:00:00',
 '20AUG2020 09:00:00',
 '20AUG2020 10:00:00',
 '20AUG2020 11:00:00',
 '20AUG2020 12:00:00',
 '20AUG2020 13:00:00',
 '20AUG2020 14:00:00',
 '20AUG2020 15:00:00',
 '20AUG2020 16:00:00',
 '20AUG2020 17:00:00',
 '20AUG2020 18:00:00',
 '20AUG2020 19:00:00',
 '20AUG2020 20:00:00',
 '20AUG2020 21:00:00',
 '20AUG2020 22:00:00',
 '20AUG2020 23:00:00',
 '21AUG2020 00:00:00',
 '21AUG2020 01:00:00',
 '21AUG2020 02:00:00',
 '21AUG2020 03:00:00',
 '21AUG2020 04:00:00',
 '21AUG2020 05:00:00',
 '21AUG2020 06:00:00',
 '21AUG2020 07:00:00',
 '21AUG2020 08:00:00',
 '21AUG2020 09:00:00',
 '21AUG2020 10:00:00',
 '21AUG2020 11:00:00',
 '21AUG2020 12:00:00',
 '21AUG2020 13:00:00',
 '21AUG2020 14:00:00',
 '21AUG2020 15:00:00',
 '21AUG2020 16:00:00',
 '21AUG2020 17:00:00',
 '21AUG2020 18:00:00',
 '21AUG2020

In [12]:
# Get nearest cell to the QAQC points
from scipy.spatial import cKDTree

def ckdnearest(gdA, gdB):

    nA = np.array(list(gdA.geometry.apply(lambda x: (x.x, x.y))))
    nB = np.array(list(gdB.geometry.apply(lambda x: (x.x, x.y))))
    btree = cKDTree(nB)
    dist, idx = btree.query(nA, k=1)
    gdB_nearest = gdB.iloc[idx].drop(columns="geometry").reset_index(drop=True)
    gdf = pd.concat(
        [
            gdA.reset_index(drop=True),
            gdB_nearest,
            pd.Series(dist, name='dist')
        ], 
        axis=1)

    return gdf

# gdf_nearest = ckdnearest(gdf_qaqc, gdf_hdf)

In [176]:
wse = hdf_file['Results/Unsteady/Output/Output Blocks/Base Output/Unsteady Time Series/Reference Points/Water Surface']
wse_data = np.array(wse)
gdf_hdf['WSE'] = wse_data.T.tolist()

gdf_hdf

,Name,geometry,WSE
0,Reference Point 200,POINT (-88.51347 30.28935),"[-48.70953369140625, 0.0877610519528389, -0.19..."
1,Reference Point 199,POINT (-89.16234 30.24164),"[-14.562239646911621, 0.12490744143724442, -0...."
2,Reference Point 198,POINT (-89.27786 29.96188),"[-1.8383821249008179, 0.30781838297843933, 0.0..."
3,Reference Point 197,POINT (-89.47514 29.73810),"[-34.0390510559082, 0.3489358723163605, 0.1275..."
4,Reference Point 196,POINT (-89.33764 29.27282),"[-47.57225036621094, 0.37746351957321167, 0.28..."
...,...,...,...
195,Reference Point 5,POINT (-90.67874 30.45343),"[-0.4185454547405243, 0.3799999952316284, 0.38..."
196,Reference Point 4,POINT (-90.46796 30.45663),"[0.7329209446907043, 0.8289499282836914, 3.491..."
197,Reference Point 3,POINT (-90.41445 30.42006),"[0.9762344360351562, 1.941579818725586, 3.9937..."
198,Reference Point 2,POINT (-89.48732 30.32246),"[0.30610278248786926, 0.3799999952316284, 0.37..."


In [177]:
# iterate through each WSE list and round the values to 2 decimal places
for i, row in gdf_hdf.iterrows():
    wse = [round(x, 2) for x in row['WSE']]
    # replace the WSE list with the rounded values in the dataframe.
    gdf_hdf.at[i, 'WSE'] = wse

gdf_hdf

,Name,geometry,WSE
0,Reference Point 200,POINT (-88.51347 30.28935),"[-48.71, 0.09, -0.19, -0.49, -0.65, -0.77, -0...."
1,Reference Point 199,POINT (-89.16234 30.24164),"[-14.56, 0.12, -0.15, -0.43, -0.59, -0.73, -0...."
2,Reference Point 198,POINT (-89.27786 29.96188),"[-1.84, 0.31, 0.07, -0.16, -0.39, -0.58, -0.71..."
3,Reference Point 197,POINT (-89.47514 29.73810),"[-34.04, 0.35, 0.13, -0.1, -0.3, -0.47, -0.57,..."
4,Reference Point 196,POINT (-89.33764 29.27282),"[-47.57, 0.38, 0.29, 0.15, 0.02, -0.07, -0.15,..."
...,...,...,...
195,Reference Point 5,POINT (-90.67874 30.45343),"[-0.42, 0.38, 0.38, 0.38, 0.38, 0.38, 0.38, 0...."
196,Reference Point 4,POINT (-90.46796 30.45663),"[0.73, 0.83, 3.49, 3.86, 4.11, 4.3, 4.39, 4.41..."
197,Reference Point 3,POINT (-90.41445 30.42006),"[0.98, 1.94, 3.99, 4.47, 4.49, 4.38, 4.22, 4.0..."
198,Reference Point 2,POINT (-89.48732 30.32246),"[0.31, 0.38, 0.38, 0.38, 0.38, 0.38, 0.38, 0.3..."


In [178]:
planName = 'LWI_Coastwide_05min'
plan_number = 'p02'

# cell_coords_data = np.array(cell_coords)
# cellIds = [item for item in range(0, len(cell_coords_data))]

In [179]:
# init plan data dictionary
plan_data_dict = {
    "planID": plan_number,
    "planName": planName,
    "datetime": list(timestamps),
    "pointData": [
        # {
        #     "name": row['id'],
        #     "wse": wse_data[:,cell]
        # }
    ]

}

In [180]:
# For each reference point, get the WSE timeseries
for i, row in gdf_hdf.iterrows():
    wse = row['WSE']   # append point data to plan data dictionary
    plan_data_dict['pointData'].append({
        "name": row['Name'],
        "wse": row['WSE']
    })

# Write the plan data dictionary to a json file
plan_data_json_output_file = f"../../output/mapByReferencePoints/{planName}_{plan_number}_data.json"
with open(plan_data_json_output_file, 'w') as f:
    json.dump(plan_data_dict, f)

In [181]:
# get the event-based *.json files from ./output
import glob
import os
event_wse_json_files = glob.glob('../../output/mapByHWM/*.json')

# Create a json file for each point
pt_json_output_dir = '../../output/mapByHWM/point_jsons'
if not os.path.exists(pt_json_output_dir):
    os.makedirs(pt_json_output_dir)

# For each HWM point, get the WSE timeseries and write to a point json file.
for i, row in ref_points_gdf.iterrows():
    pt_dict = {}
    for wse_json_file in event_wse_json_files:
        wse_json = json.load(open(wse_json_file, 'r'))
        for pointData in wse_json['pointData']:
            if pointData['name'] == row['Name']:
                planName = wse_json['planName']
                pt_dict[planName] = {}
                pt_dict[planName]['datetime'] = wse_json['datetime']
                pt_dict[planName]['wse'] = pointData['wse']
                json.dump(pt_dict, open(f"{pt_json_output_dir}/{row['Name']}.json", 'w'))
    